In [1]:
from adapt.approx_layers.systolic_utils import compare_exact_vs_approx, testlinear,check_equal
check_equal()

W0823 22:23:20.471303 140001060800320 tensor_quantizer.py:196] amax setter is not designed to take tensor.
/etc/pytorch-quantization/pytorch_quantization/nn/modules/tensor_quantizer.py:198: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('_amax', torch.tensor(value))
W0823 22:23:20.473290 140001060800320 tensor_quantizer.py:196] amax setter is not designed to take tensor.


Equal: True


True

# Model evaluation and re-training with AdaPT on Cifar10 dataset

In this notebook you can evaluate different approximate multipliers on various models based on Cifar10 dataset

Steps:
* Select models to load 
* Select number of threads to use
* Choose approximate multiplier 
* Load model for evaluation
* Load dataset
* Run model calibration for quantization
* Run model evaluation
* Run approximate-aware re-training
* Rerun model evaluation

**Note**:
* This notebook should be run on a X86 machine

* Please make sure you have run the installation steps first

In [ ]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

## Select models to load 

The weights must be downloaded in state_dicts folder.


In [ ]:
from models.resnet import resnet18, resnet34, resnet50
from models.vgg import vgg11_bn, vgg13_bn, vgg19_bn
from models.densenet import densenet121, densenet161, densenet169
from models.inception import inception_v3 # slow, propably bad cifar10 implementation of inception for PT

## Select number of threads to use

For optimal performance set them as the number of your cpu threads (not cpu cores)

In [ ]:
threads = 40
torch.set_num_threads(threads)

#maybe better performance
%env OMP_PLACES=cores
%env OMP_PROC_BIND=close
%env OMP_WAIT_POLICY=active

## Choose approximate multiplier 

Two approximate multipliers are already provided

**mul8s_acc** - (header file: mul8s_acc.h)   <--  default

**mul8s_1L2H** - (header file: mul8s_1L2H.h)



In order to use your custom multiplier you need to use the provided tool (LUT_generator) to easily create the C++ header for your multiplier. Then you just place it inside the adapt/cpu-kernels/axx_mults folder. The name of the axx_mult here must match the name of the header file. The same axx_mult is used in all layers. 

Tip: If you want explicitly to set for each layer a different axx_mult you must do it from the model definition using the respective AdaPT_Conv2d class of each layer.

In [ ]:
axx_mult = 'mul8s_acc'

## Load model for evaluation

Jit compilation method loads 'on the fly' the C++ extentions of the approximate multipliers. Then the pytorch model is loaded

In [ ]:
model = resnet50(pretrained=True, axx_mult = axx_mult)

model.eval() # for evaluation

## Load dataset


In [ ]:
def val_dataloader(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=128,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader

transform = T.Compose(
        [
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)),
        ]
    )
dataset = CIFAR10(root="datasets/cifar10_data", train=True, download=True, transform=transform)

evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)

data = val_dataloader()

# data_t is used for calibration purposes and is a subset of train-set
data_t = DataLoader(trainset_1, batch_size=128,
                                            shuffle=False, num_workers=0)


## Run model calibration for quantization

Calibrates the quantization parameters 

Need to re-run it each time the model changes

In [ ]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
    # optional - test different calibration methods
    #amax = compute_amax(model, method="mse")
    #amax = compute_amax(model, method="entropy")
    

## Run model evaluation

Tip: observe how the execution becomes faster and faster with each batch as the CPU achieves better cache re-use on the LUT table

In [ ]:
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

## Run approximate-aware re-training


## Rerun model evaluation

In [ ]:
correct = 0
total = 0
save_path = "aa_finetuned_model.pth"
device = torch.device("cpu")
model = resnet50(pretrained=False, axx_mult=axx_mult)
model.load_state_dict(torch.load(save_path, map_location="cpu"))
model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

In [ ]:
# force TensorQuantizers to load buffers on CPU instead of .cuda()
def force_quantizer_cpu():
    import torch
    from pytorch_quantization.nn import modules

    orig_fn = modules.tensor_quantizer.TensorQuantizer._load_from_state_dict

    def new_fn(self, state_dict, prefix, *args, **kwargs):
        key = prefix + '_amax'
        if key in state_dict:
            buf = state_dict[key].data.cpu()   # force CPU
            self.register_buffer("_amax", buf)
        else:
            # fallback to original
            orig_fn(self, state_dict, prefix, *args, **kwargs)

    modules.tensor_quantizer.TensorQuantizer._load_from_state_dict = new_fn

force_quantizer_cpu()

In [ ]:
#from torchvision.models import resnet50
from models.resnet import resnet18, resnet34, resnet50
from torchvision import transforms
from adapt.approx_layers.systolic_utils import compare_exact_vs_approx
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),   # CIFAR-10 stats (example)
])

test_dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)

# IMPORTANT: num_workers=0 to avoid OpenMP + fork segfaults with C++ extensions
from torch.utils.data import DataLoader
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=0, pin_memory=False)

# Your already-initialized/trained model:
# model = resnet50(pretrained=True, axx_mult=axx_mult)  # (or your own wrapper)

acc_baseline, acc_exact, acc_approx, delta = compare_exact_vs_approx(
    model,
    test_loader,
    model_factory=lambda: resnet50(pretrained=True, axx_mult = axx_mult),
    axx_mult="mul8s_acc",
    device="cpu",
)

print(f"Baseline: {acc_baseline:.4f} | Exact systolic: {acc_exact:.4f} | "
      f"Approx systolic: {acc_approx:.4f} | Δ={delta:.4f}")


In [ ]:
axx_mult = 'mul8s_1L2H'
acc_baseline, acc_exact, acc_approx, delta = compare_exact_vs_approx(
    model,
    test_loader,
    model_factory=lambda: resnet50(pretrained=True, axx_mult = axx_mult),
    axx_mult=axx_mult,
    device="cpu",
)

print(f"Baseline: {acc_baseline:.4f} | Exact systolic: {acc_exact:.4f} | "
      f"Approx systolic: {acc_approx:.4f} | Δ={delta:.4f}")